In [1]:
# from mido import MidiFile
from funcMidi import * # create by SH

MidiFileName = 'p20190719 melody.mid'


(midiArray, dicEvent) = MidiCategorize(MidiFileName)
midiArray


Track 0: Steel Guitar


[[57, 0, 240, 'MEL', 0],
 [60, 240, 480, 'MEL', 1],
 [62, 480, 720, 'MEL', 2],
 [63, 720, 960, 'MEL', 3],
 [62, 960, 1200, 'MEL', 4],
 [60, 1200, 1440, 'MEL', 5],
 [57, 1440, 1680, 'MEL', 6],
 [55, 1680, 1920, 'MEL', 7],
 [0, 2919, 2920, 'dummy line', 8]]

In [2]:
noteEvent = 0
events = []

while noteEvent in range(len(midiArray)-1):
    events.append([midiArray[noteEvent][3],midiArray[noteEvent][0:3]])
    noteEvent = noteEvent + 1
    while midiArray[noteEvent][4] == midiArray[noteEvent -1][4] and noteEvent in range(len(midiArray)):
        events[midiArray[noteEvent][4]].append(midiArray[noteEvent][0:3])
        noteEvent = noteEvent + 1
events


[['MEL', [57, 0, 240]],
 ['MEL', [60, 240, 480]],
 ['MEL', [62, 480, 720]],
 ['MEL', [63, 720, 960]],
 ['MEL', [62, 960, 1200]],
 ['MEL', [60, 1200, 1440]],
 ['MEL', [57, 1440, 1680]],
 ['MEL', [55, 1680, 1920]]]

In [3]:
dicNoteOnFingerBoard = funcCreatNoteDic()

# 6666666666666666
# need to set a function to extract continous MELODY
contMel = [x[1][0] for x in events]

possible = [] # possible positions
for note in contMel: # continuous melody
    possible.append(funcNote2Domain(note, dicNoteOnFingerBoard))

possible


capo = 0
tuning = [64, 59, 55, 50, 45, 40]
Notes in Finger board:
[64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76]
[59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]
[55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67]
[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62]
[45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57]
[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]


[[[3, 2, 1],
  [3, 2, 2],
  [3, 2, 3],
  [3, 2, 4],
  [4, 7, 1],
  [4, 7, 2],
  [4, 7, 3],
  [4, 7, 4],
  [5, 12, 1],
  [5, 12, 2],
  [5, 12, 3],
  [5, 12, 4]],
 [[2, 1, 1],
  [2, 1, 2],
  [2, 1, 3],
  [2, 1, 4],
  [3, 5, 1],
  [3, 5, 2],
  [3, 5, 3],
  [3, 5, 4],
  [4, 10, 1],
  [4, 10, 2],
  [4, 10, 3],
  [4, 10, 4]],
 [[2, 3, 1],
  [2, 3, 2],
  [2, 3, 3],
  [2, 3, 4],
  [3, 7, 1],
  [3, 7, 2],
  [3, 7, 3],
  [3, 7, 4],
  [4, 12, 1],
  [4, 12, 2],
  [4, 12, 3],
  [4, 12, 4]],
 [[2, 4, 1],
  [2, 4, 2],
  [2, 4, 3],
  [2, 4, 4],
  [3, 8, 1],
  [3, 8, 2],
  [3, 8, 3],
  [3, 8, 4]],
 [[2, 3, 1],
  [2, 3, 2],
  [2, 3, 3],
  [2, 3, 4],
  [3, 7, 1],
  [3, 7, 2],
  [3, 7, 3],
  [3, 7, 4],
  [4, 12, 1],
  [4, 12, 2],
  [4, 12, 3],
  [4, 12, 4]],
 [[2, 1, 1],
  [2, 1, 2],
  [2, 1, 3],
  [2, 1, 4],
  [3, 5, 1],
  [3, 5, 2],
  [3, 5, 3],
  [3, 5, 4],
  [4, 10, 1],
  [4, 10, 2],
  [4, 10, 3],
  [4, 10, 4]],
 [[3, 2, 1],
  [3, 2, 2],
  [3, 2, 3],
  [3, 2, 4],
  [4, 7, 1],
  [4, 7, 2],
  [4, 7, 3],

In [31]:
# Calculate melody transition cost
def funcCalMelCost(event0Possible, event1Possible):
    costMatrixUnit = []
    for pos0 in event0Possible:
        pos0beg =[]
        for pos1 in event1Possible:
            
            # cost is putting here
            # initialise cost
            costStretch = float("inf")
            costLocal = float("inf")
            # maybe write a function, input is pos0 and pos1
            costAlong  = funcCalCostAlong(pos0, pos1)
            costAcross = funcCalCostAcross(pos0, pos1)
            cost = costAlong + costAcross

            pos0beg.append(cost)
        costMatrixUnit.append(pos0beg)
    return costMatrixUnit

In [82]:
# 這個先保留用np array來儲存的方式 
# 現在先用下面那個list的來測試
event0Possible = possible[2]
event1Possible = possible[3]

costBest = []
for i in range(len(possible)-1):
    event0Possible = possible[i]
    event1Possible = possible[i+1]
    
    costMatrixUnit = funcCalMelCost(event0Possible, event1Possible)
    
    npCostMatrixUnit = np.array(costMatrixUnit)
    minValue = np.min(npCostMatrixUnit, axis = 0)
    minIndex = np.argmin(npCostMatrixUnit, axis = 0)
#     minSummary = np.vstack
    costBest.append(np.vstack([minValue,minIndex]))
costBest[0]

array([[ 1.75,  1.75,  1.75,  6.  ,  4.  ,  4.  ,  4.25,  2.5 ,  6.5 ,
         5.5 ,  5.5 ,  5.  ],
       [ 1.  ,  2.  ,  3.  ,  1.  ,  6.  ,  7.  ,  0.  ,  0.  , 10.  ,
         0.  ,  0.  ,  4.  ]])

In [17]:
for i in range(np.shape(costBest[-1])[1]) :
    

(2, 9)

In [108]:
# event0Possible = possible[2]
# event1Possible = possible[3]

costBest = []
iniSize = len(possible[0])
npPreMinValue = np.zeros(iniSize)
preMinIndex = [[0] for i in range(iniSize)]
# for i in range(1):
for i in range(len(possible)-1):
    event0Possible = possible[i]
    event1Possible = possible[i+1]
    
    costMatrixUnit = funcCalMelCost(event0Possible, event1Possible)
    
    npCostMatrixUnit = np.array(costMatrixUnit)
    npMinValue = np.min(npCostMatrixUnit, axis = 0)
        
    npMinIndex = np.argmin(npCostMatrixUnit, axis = 0)
    minIndex = list(npMinIndex)
    
    
    npCombMinValue = np.zeros(len(npMinValue))
#     combMinIndex = [0]*len(npMinValue)
    combMinIndex = [0 for icmi in range(len(npMinValue))]
#     print(combMinIndex)
    for imin in range(len(npMinValue)):
        npCombMinValue[imin] = npMinValue[imin] + npPreMinValue[npMinIndex[imin]]
        combMinIndex[imin] = copy.deepcopy(preMinIndex[npMinIndex[imin]])
        combMinIndex[imin].append(copy.deepcopy(minIndex[imin]))
    npPreMinValue = npCombMinValue
    preMinIndex = combMinIndex

#     minSummary = np.vstack
    costBest.append(copy.deepcopy([npCombMinValue,combMinIndex]))
# costBest

In [109]:
for choose in range(len(costBest[-1][1])):
#     choose = -1
    bestPathNumber = costBest[-1][1][choose][1:]
    bestPath = []
    for i in range(len(bestPathNumber)):
        bestPath.append(possible[i][bestPathNumber[i]])
    bestPath.append(possible[-1][choose])
    print("cost=" + str(costBest[-1][0][choose]) + str(bestPath))
# bestPathNumber
# bestPath


cost=13.25[[3, 2, 2], [2, 1, 1], [2, 3, 3], [2, 4, 4], [2, 3, 3], [2, 1, 1], [3, 2, 2], [3, 0, 0]]
cost=28.25[[4, 7, 3], [3, 5, 1], [3, 7, 3], [3, 8, 4], [3, 7, 3], [3, 5, 1], [4, 7, 3], [4, 5, 1]]
cost=26.75[[4, 7, 4], [3, 5, 2], [3, 7, 4], [2, 4, 1], [3, 7, 4], [3, 5, 2], [4, 7, 4], [4, 5, 2]]
cost=20.25[[3, 2, 1], [3, 5, 4], [2, 3, 2], [2, 4, 3], [2, 3, 2], [3, 5, 4], [3, 2, 1], [4, 5, 3]]
cost=18.75[[3, 2, 1], [3, 5, 4], [2, 3, 2], [2, 4, 3], [2, 3, 2], [3, 5, 4], [3, 2, 1], [4, 5, 4]]
cost=22.25[[3, 2, 2], [2, 1, 1], [2, 3, 3], [2, 4, 4], [2, 3, 3], [2, 1, 1], [5, 12, 3], [5, 10, 1]]
cost=21.5[[3, 2, 1], [3, 5, 4], [2, 3, 2], [2, 4, 3], [2, 3, 2], [3, 5, 4], [3, 2, 1], [5, 10, 2]]
cost=21.5[[3, 2, 1], [3, 5, 4], [2, 3, 2], [2, 4, 3], [2, 3, 2], [3, 5, 4], [3, 2, 1], [5, 10, 3]]
cost=34.75[[4, 7, 1], [4, 10, 4], [3, 7, 1], [3, 8, 2], [3, 7, 1], [4, 10, 4], [4, 7, 1], [5, 10, 4]]


In [53]:
npMinIndex = np.argmin(npCostMatrixUnit, axis = 0)
minIndex = list(npMinIndex)
npCombMinValue = np.zeros(len(npMinValue))
combMinIndex = [0]*len(npMinValue)

for imin in range(len(npMinValue)):
    npCombMinValue[imin] = npMinValue + npPreMinValue[npMinIndex[imin]]
    combMinIndex[imin] = preMinIndex[npMinIndex[imin]]
    combMinIndex[imin].append(npMinIndex[imin])
    


1
2
3
1
6
7
0
0
10
0
0
4


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]